In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import pickle as pkl
import sys

import warnings
warnings.simplefilter("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import cv2

yolov5 = torch.hub.load("ultralytics/yolov5", "custom", "yolov5/weights/yolov5s.pt")

videos_dir = "data/raw"
filename = "20240602 220107 [OyLe-Z-knPU].mp4"
filepath = os.path.join(videos_dir, filename)

capture = cv2.VideoCapture(filepath)
num_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

Using cache found in C:\Users\Clifford_PP/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-4 Python-3.10.16 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
# from tqdm import trange

# from src.utils import plot_image, IMAGE_SIZE

# frame_num = 1
# ball_locations = pd.DataFrame()

# batch_size = 240
# batch = []

# pbar = trange(num_frames)
# while capture.isOpened():
#     pbar.update(1)
#     pbar.set_description("Batching")

#     ret, frame = capture.read()
#     if not ret:
#         break

#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     batch.append(frame_rgb)

#     if (len(batch) < batch_size) and (len(batch) + frame_num < num_frames):
#         continue

#     pbar.set_description("Inference")
#     results = yolov5(batch)

#     for i, res_df in enumerate(results.pandas().xyxy):
#         if "sports ball" not in res_df["name"].values:
#             continue

#         curr_frame_num = frame_num + i
#         res_df["frame_num"] = curr_frame_num
#         ball_loc = res_df[res_df["name"] == "sports ball"]
#         ball_locations = pd.concat([ball_locations, ball_loc])
    
#     frame_num += batch_size
#     batch = []

# ball_locations["filename"] = filename
processed_dir = os.path.join("data/processed")
# ball_locations = ball_locations.reset_index(drop = True)
# ball_locations.to_csv(os.path.join(processed_dir, "ball_locations.csv"))

ball_locations = pd.read_csv(os.path.join(processed_dir, "ball_locations.csv"), index_col = 0)
ball_locations.head()

,xmin,ymin,xmax,ymax,confidence,class,name,frame_num,filename
0,509.142578,172.461655,518.546875,180.298965,0.454018,32,sports ball,1,20240602 220107 [OyLe-Z-knPU].mp4
1,509.150391,172.476807,518.542725,180.277954,0.453389,32,sports ball,2,20240602 220107 [OyLe-Z-knPU].mp4
2,509.150421,172.477158,518.543091,180.278397,0.453604,32,sports ball,3,20240602 220107 [OyLe-Z-knPU].mp4
3,509.150421,172.477158,518.543091,180.278397,0.453604,32,sports ball,4,20240602 220107 [OyLe-Z-knPU].mp4
4,509.150421,172.477158,518.543091,180.278397,0.453604,32,sports ball,5,20240602 220107 [OyLe-Z-knPU].mp4


Yolov5 is not accurate enough to pickup the position of the ball. Will need to finetune with respect to the dataset we have.

In [5]:
# Find the mean color of each captured bbox
from src.utils import plot_image

capture = cv2.VideoCapture(filepath)
frame_num = 1

potential_labels = []
savedir = "data/processed/potential_labels"
while capture.isOpened():
    ret, frame = capture.read()

    if ret is None:
        break

    bbox_locations = ball_locations[ball_locations["frame_num"] == frame_num]
    for idx in range(len(bbox_locations)):
        location = bbox_locations.iloc[idx, :]
        x1, y1, x2, y2 = location[["xmin", "ymin", "xmax", "ymax"]].astype(int)

        cutout = frame[y1: y2, x1: x2]
        plot_image(cutout)
        plt.savefig(os.path.join(savedir, f"{frame_num}_{idx}.jpg"))

        potential_labels.append(cutout)

    frame_num += 1
    if frame_num >= 10000:
        break

In [38]:
# Perform clustering on the yellowness of an image. 
# Yellow is a combination of red and green, so cluster based on the mean of
# these 2 values

def get_average_color(frame):
    return frame.mean(axis = 0).mean(axis = 0)

average_colors = [get_average_color(frame) for frame in potential_labels]
len(average_colors)

4158

In [41]:
from sklearn.cluster import KMeans

clusterer = KMeans(n_clusters = 5)
cluster_preds = clusterer.fit_predict(average_colors)

cluster_num, cluster_counts = np.unique(cluster_preds, return_counts = True)
cluster_counts

array([1359,  691,  612,  964,  532])